# makemore: part 3

#### Creating the dataset and i2s <-> s2i

In [10]:
# Import libraries 
import math 
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt         # for making figures
%matplotlib inline
from typing import Union, Dict, Tuple, List 

In [11]:
# Here we will import the list with all the words/names 
import os 
words_txt_file_path = os.path.join(os.path.expanduser("~"), "NN_zero_to_hero", "Lectures", "Makemore_repo", "names.txt")
words = open(words_txt_file_path, 'r').read().splitlines()

print(f"Now we have read the {len(words)} words from the file. Here we display the first 8 words\n{words[:8]}")

Now we have read the 32033 words from the file. Here we display the first 8 words
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [12]:
### Here we are building the vocabulary and mapping from char to integer and vice versa 
chars = sorted(list(set(''.join(words))))           # Read all unique characters in all words 
s_to_i = {s:i+1 for i,s in enumerate(chars)}        # Create a mapping from char to index integer
s_to_i['.'] = 0                                     # Assign the index of our special start/end token '.' to 0
i_to_s = {i:s for s,i in s_to_i.items()}            # Reverse the mapping from integer to char 
num_classes = len(s_to_i)                           # Read the different number of characters available in our dataset 
vocab_size = num_classes                            # Vocabulary = number of different tokens available for the model 

print(f"Now we have created the mappings between chars and integers and vice versa, which yields to {num_classes} different, possible characters (i.e. classes) in our dataset:\n{s_to_i=}\n{i_to_s=}")

Now we have created the mappings between chars and integers and vice versa, which yields to 27 different, possible characters (i.e. classes) in our dataset:
s_to_i={'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
i_to_s={1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [13]:
### Build the dataset

# Create the dataset arrays from the words list
def build_dataset(words, s2i: Dict = s_to_i, block_size: int = 3, num_words: Union[int, None] = None, ):  
  X, Y = [], []
  for w in (words if num_words is None else words[:num_words] if isinstance(words[:num_words], list) else [words[:num_words]]): 
    context = [0] * block_size
    for ch in w + '.':
      ix = s2i[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]                              # crop and append the next character to the context
  return torch.tensor(X), torch.tensor(Y)


# Print metadata 
random.seed(42)                                                 # Choose a seed for deterministic shuffling 
random.shuffle(words)                                           # inplace shuffling of the words list 
n1 = int(0.8*len(words))                                        # Extract an integer for the 80% of the dataset 
n2 = int(0.9*len(words))                                        # Extract an integer for the 90% of the dataset 

block_size = 3                                                  # context length: how many characters do we take to predict the next one?
Xtr,  Ytr  = build_dataset(words[:n1], block_size=block_size)   # Build the training split 
Xdev, Ydev = build_dataset(words[n1:n2], block_size=block_size) # Build the validation (dev) split 
Xte,  Yte  = build_dataset(words[n2:], block_size=block_size)   # Build the test split 


#### MLP revisited

##### Functions for generating model parameters and training the model

In [14]:
##### Generate parameters function 

# Generate the parameters of the model 
def generate_parameters(random_seed: int = 2147483648, emb_dim: int = 2, num_hidden_neurons: int = 100, block_size: int = 3, num_classes: int = num_classes, 
                        return_generator: bool = False, emb_scale: float = 1.0, W1_scale: float = 1.0, W2_scale: float = 1.0, b1_scale: float = 1.0, b2_scale: float = 1.0,
                        emb_offset: float = 0.0, W1_offset: float = 0.0, W2_offset: float = 0.0, b1_offset: float = 0.0, b2_offset: float = 0.0, use_BN: bool = True) -> Tuple[torch.tensor]:
    g = torch.Generator().manual_seed(random_seed)          # for reproducibility and deterministic generation 
    
    # Create all the layers and trainable parameters. Scale them according to the chosen hyperparameters parameters
    C = torch.randn((num_classes, emb_dim), generator=g) * emb_scale + emb_offset
    W1 = torch.randn((emb_dim*block_size, num_hidden_neurons), generator=g) * W1_scale + W1_offset 
    b1 = torch.randn(num_hidden_neurons, generator=g) * b1_scale + b1_offset 
    W2 = torch.randn((num_hidden_neurons, num_classes), generator=g) * W2_scale + W2_offset 
    b2 = torch.randn(num_classes, generator=g) * b2_scale + b2_offset 

    # BatchNorm parameters
    bn_gain = torch.ones((1, num_hidden_neurons))                       # Initiate the running gain to be ones 
    bn_bias = torch.zeros((1, num_hidden_neurons))                      # Initiate the running bias to be zeros 
    bn_mean_running = torch.zeros((1, num_hidden_neurons))              # Initiate the running mean to be zeros 
    bn_std_running = torch.ones((1, num_hidden_neurons))                # Initiate the running std to be ones 

    # Assure all parameters have set "required_grad" to True and return the parameters 
    for p in [C, W1, b1, W2, b2] if not use_BN else [C, W1, b1, W2, b2, bn_gain, bn_bias]:
        p.requires_grad = True
    return_list = [C, W1, b1, W2, b2]
    if use_BN:
        return_list.extend([bn_gain, bn_bias, bn_mean_running, bn_std_running])
    if return_generator:
        return_list.append(g)
    return tuple(return_list)


# Compute the total number of parameters in our network 
C, W1, b1, W2, b2, bn_gain, bn_bias, bn_mean_running, bn_std_running = generate_parameters()
parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
num_parameters = sum(p.nelement() for p in parameters)          # number of parameters in total
print(f"This means that with the embedding dimension {C.shape[1]} and a two layer MLP with {W1.shape[1]} in each layer, we end up at {num_parameters} trainable parameters")

This means that with the embedding dimension 2 and a two layer MLP with 100 in each layer, we end up at 3681 trainable parameters


In [15]:
##### Define the training function

# This we will use to both showcase three examples:
# 1) How to perform a learning rate search? i.e. we will search for the proper learning rate 
# 2) What happens when we overfit with just the 32 sequences from our X array, i.e. to check that the model is capable of learning
# 3) Training the model on the entire dataset 



def train_func(inp_data: torch.tensor, targets: torch.tensor, emb_matrix: torch.tensor = C, batch_size: int = 32, num_iterations: int = 20,
        lr_scheduler: Union[torch.tensor, None] = None, lr: Union[torch.tensor, None] = 1e-4, regenerate_parameters: bool = True, use_batch_norm: bool = False, **kwargs) -> Tuple[List, List, List, List]:
    # Initiating lists to store the data that we want to keep 
    lri = []
    lossi = []
    stepi = []

    # Regenerate the parameters of the network at random state 
    emb_matrix, W1, b1, W2, b2, g_and_BN = generate_parameters(return_generator=True, use_BN=use_batch_norm, **kwargs)
    parameters = [emb_matrix, W1, b1, W2, b2]
    if use_batch_norm:
        g, bn_gain, bn_bias, bn_mean_running, bn_std_running = g_and_BN
        parameters.extend([bn_gain, bn_bias])
    else:
        g = g_and_BN 

    # Iterating through all the selected iterations 
    if lr_scheduler is None:
        lr_scheduler = torch.full((num_iterations,), lr)                        # Repeat the provided learning rate for all iterations 
    for i in range(num_iterations):
        ### Forward pass 
        # Create a single mini batch from the dataset 
        idx = torch.randint(0, inp_data.shape[0], (batch_size,), generator=g)   # Create integer index array for creating stochastic mini-batches 
        emb = emb_matrix[inp_data[idx]]                                         # (32, 3, 10) --> Read the embedding vectors of the randomly selected examples for the current iteration 
        emb_cat = emb.view(-1, emb.shape[1]*emb.shape[2])                       # Concatenate the last 'block_size' characters together in an array of sequences 

        # Linear layer 
        h_pre_act_func = emb_cat @ W1 + b1                                      # (32, 200)   --> Compute the output of the first hidden layer of our MLP model 

        # BatchNorm layer
        # -------------------------------------------------------------
        if use_batch_norm:
            bn_mean_i = h_pre_act_func.mean(0, keepdim=True) 
            bn_std_i = h_pre_act_func.std(0, keepdim=True)
            h_pre_act_func = bn_gain * (h_pre_act_func-bn_mean_i) / bn_std_i + bn_bias 

            with torch.no_grad():
                bn_mean_running = 0.999 * bn_mean_running + 0.001 * bn_mean_i
                bn_std_running = 0.999 * bn_std_running + 0.001 * bn_std_i 

        # Non-linearity 
        h_act_func = torch.tanh(h_pre_act_func)                                 # (32, 200)   --> Compute the
        logits = h_act_func @ W2 + b2                                                    # (32, 27)    --> Compute the logits of the forward pass 
        loss = F.cross_entropy(logits, targets[idx])                            # Compute the NLL loss (equal to the cross entropy) using the efficient pytorch implementation 

        
        ### Backward pass 
        # According to Karparthy, setting the gradient to None is more efficient than p.grad = 0, when using PyTorch .backward() method 
        for p in parameters:
            p.grad = None
        loss.backward()                                 # Compute the new gradients for all parameters 

        # Update all the parameters 
        lr = lr_scheduler[i]
        for p in parameters:
            p.data += -lr *p.grad 
        
        # Track the stats
        lri.append(lr)
        stepi.append(i)
        lossi.append(loss.log10().item())
    
    # Return the lists of stats
    return lri, stepi, lossi, parameters

##### Train again, with different setups 

In [ ]:
# MLP revisited
n_embd = 10                 # the dimensionality of the character embedding vectors
n_hidden = 200              # the number of neurons in the hidden layer of the MLP


# Generate initial parameters for the model 
C, W1, b1, W2, b2 = generate_parameters(emb_dim=10, num_hidden_neurons=200)


g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5) #* 0.2
#b1 = torch.randn(n_hidden,                        generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0


# BatchNorm parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

In [ ]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)
  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  

In [ ]:
plt.plot(lossi)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 # + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnstd = hpreact.std(0, keepdim=True)


In [ ]:
#### The split loss will be computed without gradients 
## The decorator will make these computations much more efficient, as we are telling PyTorch to not create computational graphs under the hood, for later gradient calculations, as we are not going to use them ... 

@torch.no_grad()        # this decorator disables gradient tracking => similar to "with torch.no_grad():" ...
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 # + b1
  #hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
  hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

## loss log

### original:
train 2.1245384216308594
val   2.168196439743042

### fix softmax confidently wrong:
train 2.07
val   2.13

### fix tanh layer too saturated at init:
train 2.0355966091156006
val   2.1026785373687744

### use semi-principled "kaiming init" instead of hacky init:
train 2.0376641750335693
val   2.106989622116089

### add batch norm layer
train 2.0668270587921143
val 2.104844808578491


In [ ]:
# SUMMARY + PYTORCHIFYING -----------

In [ ]:
# Let's train a deeper network
# The classes we create here are the same API as nn.Module in PyTorch

class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

C = torch.randn((vocab_size, n_embd),            generator=g)
layers = [
  Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
# layers = [
#   Linear(n_embd * block_size, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, vocab_size),
# ]

with torch.no_grad():
  # last layer: make less confident
  layers[-1].gamma *= 0.1
  #layers[-1].weight *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

In [ ]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function
  
  # backward pass
  for layer in layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

  if i >= 1000:
    break # AFTER_DEBUG: would take out obviously to run full optimization

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' % (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('activation distribution')

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out.grad
    print('layer %d (%10s): mean %+f, std %e' % (i, layer.__class__.__name__, t.mean(), t.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('gradient distribution')

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i,p in enumerate(parameters):
  t = p.grad
  if p.ndim == 2:
    print('weight %10s | mean %+f | std %e | grad:data ratio %e' % (tuple(p.shape), t.mean(), t.std(), t.std() / p.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'{i} {tuple(p.shape)}')
plt.legend(legends)
plt.title('weights gradient distribution');

In [ ]:
plt.figure(figsize=(20, 4))
legends = []
for i,p in enumerate(parameters):
  if p.ndim == 2:
    plt.plot([ud[j][i] for j in range(len(ud))])
    legends.append('param %d' % i)
plt.plot([0, len(ud)], [-3, -3], 'k') # these ratios should be ~1e-3, indicate on plot
plt.legend(legends);


In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  x = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, y)
  print(split, loss.item())

# put layers into eval mode
for layer in layers:
  layer.training = False
split_loss('train')
split_loss('val')

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])] # (1,block_size,n_embd)
      x = emb.view(emb.shape[0], -1) # concatenate the vectors
      for layer in layers:
        x = layer(x)
      logits = x
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(i_to_s[i] for i in out)) # decode and print the generated word

In [ ]:
# DONE; BONUS content below, not covered in video

In [ ]:
# BatchNorm forward pass as a widget

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.stats as stats
import numpy as np

def normshow(x0):
  
  g = torch.Generator().manual_seed(2147483647+1)
  x = torch.randn(5, generator=g) * 5
  x[0] = x0 # override the 0th example with the slider
  mu = x.mean()
  sig = x.std()
  y = (x - mu)/sig

  plt.figure(figsize=(10, 5))
  # plot 0
  plt.plot([-6,6], [0,0], 'k')
  # plot the mean and std
  xx = np.linspace(-6, 6, 100)
  plt.plot(xx, stats.norm.pdf(xx, mu, sig), 'b')
  xx = np.linspace(-6, 6, 100)
  plt.plot(xx, stats.norm.pdf(xx, 0, 1), 'r')
  # plot little lines connecting input and output
  for i in range(len(x)):
    plt.plot([x[i],y[i]], [1, 0], 'k', alpha=0.2)
  # plot the input and output values
  plt.scatter(x.data, torch.ones_like(x).data, c='b', s=100)
  plt.scatter(y.data, torch.zeros_like(y).data, c='r', s=100)
  plt.xlim(-6, 6)
  # title
  plt.title('input mu %.2f std %.2f' % (mu, sig))

interact(normshow, x0=(-30,30,0.5))


In [ ]:
# Linear: activation statistics of forward and backward pass

g = torch.Generator().manual_seed(2147483647)

a = torch.randn((1000,1), requires_grad=True, generator=g)          # a.grad = b.T @ c.grad
b = torch.randn((1000,1000), requires_grad=True, generator=g)       # b.grad = c.grad @ a.T
c = b @ a
loss = torch.randn(1000, generator=g) @ c
a.retain_grad()
b.retain_grad()
c.retain_grad()
loss.backward()
print('a std:', a.std().item())
print('b std:', b.std().item())
print('c std:', c.std().item())
print('-----')
print('c grad std:', c.grad.std().item())
print('a grad std:', a.grad.std().item())
print('b grad std:', b.grad.std().item())

In [ ]:
# Linear + BatchNorm: activation statistics of forward and backward pass

g = torch.Generator().manual_seed(2147483647)

n = 1000
# linear layer ---
inp = torch.randn(n, requires_grad=True, generator=g)
w = torch.randn((n, n), requires_grad=True, generator=g) # / n**0.5
x = w @ inp
# bn layer ---
xmean = x.mean()
xvar = x.var()
out = (x - xmean) / torch.sqrt(xvar + 1e-5)
# ----
loss = out @ torch.randn(n, generator=g)
inp.retain_grad()
x.retain_grad()
w.retain_grad()
out.retain_grad()
loss.backward()

print('inp std: ', inp.std().item())
print('w std: ', w.std().item())
print('x std: ', x.std().item())
print('out std: ', out.std().item())
print('------')
print('out grad std: ', out.grad.std().item())
print('x grad std: ', x.grad.std().item())
print('w grad std: ', w.grad.std().item())
print('inp grad std: ', inp.grad.std().item())